Progetto di Linguistica Computazionale di Giacomo Fidone (531668) –  Estrazione di informazioni da animal_farm.txt

# Imports

In [1]:
import nltk
from nltk import ngrams
import math

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Estrazione di Informazioni da un testo

In [3]:
class Analyze:

    def __init__(self, path):
        '''Definisce una classe per l'estrazione di informazioni da un testo.'''

        self.text = self.read_file(path)
        self.sents = nltk.tokenize.sent_tokenize(self.text)
        self.tokenized_sents = [nltk.word_tokenize(sent) for sent in self.sents]
        self.tokens = [token for sent in self.tokenized_sents for token in sent]
        self.freq_dist = nltk.FreqDist(self.tokens)
        self.pos = nltk.tag.pos_tag(self.tokens, tagset='universal') # uso il tagset di UD
        self.only_pos = [pos for token, pos in self.pos]

    def top_k_words(self, k, pos_tags=None):
        '''Mostra i k token più frequenti (e relativa frequenza) in ordine decrescente. E' possibile indicare i pos-tags da
        considerare assegnando a "pos_tags" la lista di UD pos tags. '''

        if pos_tags==None:
            freqs = self.freq_dist
        else:
            tokens = [token for token, pos in self.pos if pos in pos_tags]
            freqs = nltk.FreqDist(tokens)

        top_k = {token : freqs[token] for token in list(freqs)[:k]}
        for i, token in enumerate(top_k):
            print(f'{i+1}. {token} : {top_k[token]}')

    def top_k_ngrams(self, k, n, pos=False):
        '''Mostra i primi k n-grammi di parole più frequenti (e relative frequenze) in ordine decrescente. Se "pos"=True,
        considera n-grammi di pos.'''

        if pos:
            n_grams = list(ngrams(self.only_pos, n))
        else:
            n_grams = list(ngrams(self.tokens, n))
        voc = list(set(n_grams))
        freqs = nltk.FreqDist(n_grams)

        top_k = {n_gram : freqs[n_gram] for n_gram in list(freqs)[:k]}
        for i, n_gram in enumerate(top_k):
            print(f'{i+1}. {n_gram} : {top_k[n_gram]}')

    def top_k_bigrams(self, k, pos_tags, order='frequency', display_output=True):
        '''Mostra i primi k bigrammi di parole, le cui pos corrispondono a "pos_tags", in base a "order". Valori possibili di "order" sono: "frequency" (ordinati per frequenza – default);
        "cond_prob" (ordinati per probabilità condizionata), "joint_prob" (ordinati per probabilità congiunta), "mi" (ordinati per mutual information),
        "lmi" (ordinati per local mutual information). Il metodo è usato internamente da self.common_mi_lmi(), che disattiva il flag "display_output".'''

        bigrams = list(ngrams(self.tokens, 2))
        pos_bigrams = list(ngrams(self.only_pos, 2))
        bigrams = [bigram for bigram, pos_bigram in zip(bigrams, pos_bigrams) if pos_bigram==pos_tags] # seleziono i bigrammi le cui pos corrispodono a "pos_tags"
        bigrams_freq_dist = nltk.FreqDist(bigrams)

        results = dict() # dizionario della forma {bigramma:valore}, dove valore è il valore della misura selezionata per "order"
        if order == 'frequency':
            results = bigrams_freq_dist
        else:
            voc = list(set(bigrams)) # vocabolario di bigrammi
            for bigram in voc:
                bigram_freq = bigrams_freq_dist[bigram]
                first_token_freq = self.freq_dist[bigram[0]]
                corpus_size = len(self.tokens)
                if order == 'cond_prob' or order=='joint_prob':
                    cond_prob = bigram_freq / first_token_freq # P(B|A) = F(A, B) / F(A)
                    if order == 'cond_prob':
                        results[bigram] = cond_prob
                    else:
                        results[bigram] = (first_token_freq/corpus_size) * cond_prob # P(A, B) = P(A)P(B|A)
                else:
                    first_token_prob = first_token_freq / corpus_size
                    second_token_prob = self.freq_dist[bigram[1]] / corpus_size
                    bigram_prob = bigram_freq / len(bigrams)
                    mi = math.log(bigram_prob / (first_token_prob * second_token_prob), 2) # MI = log_2(P(A, B) / P(A) * P(B))
                    if order == 'mi':
                        results[bigram] = mi
                    elif order == 'lmi':
                        results[bigram] = bigram_freq * mi # LMI = F(A, B) * MI
                    else:
                        raise ValueError(f'Invalid value for "order". Expected one of ["frequency", "cond_prob", "joint_prob", "mi", "lmi"], got "{order}".')
            results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True)) # ordino i bigrammi in senso decrescente secondo il valore assegnato

        top_k = {bigram : results[bigram] for bigram in list(results)[:k]} # considero solo i primi k bigrammi
        if display_output:
            for i, bigram in enumerate(top_k):
                print(f'{i+1}. {bigram} : {top_k[bigram]}')
        else:
            return top_k.keys()

    def common_mi_lmi(self, k, pos_tags, display_output=False):
        '''Mostra i bigrammi in comune ai primi k bigrammi (le cui pos corrispondono a 'pos_tags') ordinati per mutual information e local mutual information.'''

        mi_bigrams = self.top_k_bigrams(k, pos_tags, order='mi', display_output=False)
        lmi_bigrams = self.top_k_bigrams(k, pos_tags, order='lmi', display_output=False)
        common = [bigram for bigram in mi_bigrams if bigram in lmi_bigrams]
        if common:
            print(f'Bigrammi in comune nei primi {k} bigrammi con pos {pos_tags} ordinati per mutual information e local mutual information:')
            for bigram in common:
                print(bigram)
        else:
            print(f'Non ci sono bigrammi in comune nei primi {k} bigrammi con pos {pos_tags} ordinati per mutual information e local mutual information.')

    def analyze_sentences(self):
        '''Mostra la frase con la media della distribuzione di frequenza massima, la frase con la media di distribuzione di frequenza minima
        e la frase con la più alta probabilità secondo un modello di markov di ordine 2. Considera solo frasi la cui lunghezza è compresa tra 10 e 20 token
        e di cui almeno metà dei token non è un hapax.'''

        target_sents = [sent for sent in self.tokenized_sents if len(sent) <=20 and len(sent) >= 10] # considero solo frasi la cui lunghezza è compresa tra 10 e 20
        for sent in target_sents:
            no_hapax_words = len([token for token in sent if self.freq_dist[token] > 1]) # calcolo il numero di parole della frase con frequenza maggiore di 1
            if no_hapax_words < (len(sent) // 2): # rimuovo la frase se il numero di parole che non sono hapax è inferiore alla metà del numero di parole totale
                target_sents.remove(sent)

        max_avg_freq = 0
        max_avg_freq_sent = str()
        min_avg_freq = float('inf')
        min_avg_freq_sent = str()
        max_markov_prob = 0
        max_markov_prob_sent = str()

        for sent in target_sents:
            token_freqs = [self.freq_dist[token] for token in sent]
            avg_freq = sum(token_freqs) / len(sent) # media della distribuzione di frequenza
            markov_prob = self.markov2(sent) # probabilità secondo modello di markov di ordine 2
            if avg_freq > max_avg_freq:
                max_avg_freq = avg_freq
                max_avg_freq_sent = sent
            if avg_freq < min_avg_freq:
                min_avg_freq = avg_freq
                min_avg_freq_sent = sent
            if markov_prob > max_markov_prob:
                max_markov_prob = markov_prob
                max_markov_prob_sent = sent

        max_avg_freq_sent = self.sents[self.tokenized_sents.index(max_avg_freq_sent)] # recupero le frasi non tokenizzate per la stampa a video
        min_avg_freq_sent = self.sents[self.tokenized_sents.index(min_avg_freq_sent)]
        max_markov_prob_sent = self.sents[self.tokenized_sents.index(max_markov_prob_sent)]

        print(f'Frase con la media della distribuzione di frequenza dei token più alta ({round(max_avg_freq, 2)}): "{max_avg_freq_sent}".', )
        print(f'Frase con la media della distribuzione di frequenza dei token più bassa ({round(min_avg_freq, 2)}): "{min_avg_freq_sent}".', )
        print(f'Frase con la probabilità più alta secondo un modello di Markov di ordine 2 ({max_markov_prob}): "{max_markov_prob_sent}".', )


    def markov2(self, tokenized_sent):
        '''Restituisce la probabilità di tokenized_sent secondo un modello di markov di ordine 2
        (con add-1 smoothing) costruito a partire dal corpus di input.'''

        len_voc = len(list(set(self.tokens)))
        prob = (self.freq_dist[tokenized_sent[0]] + 1)/ (len(self.tokens) + len_voc) # P(A_1)
        prob *= self.cond_prob((tokenized_sent[0], tokenized_sent[1])) # P(A_2|A_1)

        sent_trigrams = list(ngrams(tokenized_sent, 3))
        for trigram in sent_trigrams:
            prob *= self.cond_prob(trigram) # P(A_n|A_{n-1}, A_{n-2})
        return prob

    def cond_prob(self, n_gram):
        '''Dato un n-gramma (A_1, ..., A_n), restituisce la probabilità condizionata
        P(A_n | A_1, ..., A_{n - 1}) calcolata con add-1 smoothing.'''

        n = len(n_gram)
        n_grams = list(ngrams(self.tokens, n))
        prefix = tuple(n_gram[:-1]) # contesto
        n_minus_one_grams = list(ngrams(self.tokens, n - 1))
        prefix_freq = n_minus_one_grams.count(prefix) # frequenza del contesto
        len_voc = len(list(set(self.tokens)))
        return (n_grams.count(n_gram) + 1) / (prefix_freq + len_voc) # (F(A_1, .., A_n) + 1) / (F(A_1, ..., A_{n-1}) + |V|)

    def ner(self, k):
        '''Estrae le NE dal testo e mostra, per ciascuna classe, i k token più frequenti
        (con relativa frequenza) ordinati per frequenza decrescente.'''

        ne_tree = nltk.ne_chunk(nltk.tag.pos_tag(self.tokens)) # per NER uso le POS di Penn Treebank
        ne = dict() # dizionario della forma {entity_type:[entities]}
        for node in ne_tree: # accedo a ciascun nodo dell'albero
            if hasattr(node, 'label'): # se il nodo corrisponde ad una NE, accedo alla classe di NE e alla NE
                    entity_type = node.label()
                    entity = " ".join([token for token, pos in node.leaves()])
                    if entity_type in ne.keys():
                        ne[entity_type].append(entity)
                    else:
                        ne[entity_type] = [entity]

        for entity_type, entities in ne.items():
            print(entity_type)
            freq_dist = nltk.FreqDist(entities)
            top_k = {entity : freq_dist[entity] for entity in list(freq_dist)[:k]} # considero solo i primi k elementi
            for i, entity in enumerate(top_k):
                print(f'{i+1}. {entity} : {top_k[entity]}')
            print('')

    @staticmethod
    def read_file(path_to_file):
        '''Apre un file e lo restituisce come stringa.'''

        with open (path_to_file, 'r', encoding='utf-8') as infile:
            return infile.read()

In [4]:
analyze = Analyze('animal_farm.txt')

In [5]:
analyze.top_k_words(50, pos_tags=['NOUN']) # top 50 sostantivi più frequenti

1. animals : 243
2. Napoleon : 165
3. Snowball : 126
4. farm : 109
5. Boxer : 92
6. pigs : 91
7. Jones : 80
8. time : 76
9. windmill : 69
10. Squealer : 66
11. Farm : 55
12. work : 53
13. dogs : 50
14. day : 47
15. Animal : 46
16. Mr. : 44
17. animal : 44
18. Clover : 42
19. moment : 42
20. comrades : 39
21. days : 38
22. men : 35
23. farmhouse : 34
24. Frederick : 34
25. way : 33
26. legs : 33
27. Benjamin : 32
28. sheep : 29
29. England : 28
30. Comrade : 28
31. Pilkington : 28
32. round : 27
33. end : 27
34. night : 25
35. side : 25
36. yard : 25
37. barn : 25
38. others : 24
39. year : 23
40. beings : 23
41. hens : 22
42. life : 22
43. pig : 21
44. nothing : 21
45. Rebellion : 21
46. Mollie : 20
47. week : 20
48. horses : 19
49. food : 19
50. eyes : 19


In [6]:
analyze.top_k_words(50, pos_tags=['ADV']) # top 50 avverbi più frequenti

1. not : 183
2. when : 99
3. now : 78
4. so : 46
5. never : 46
6. very : 43
7. even : 41
8. then : 38
9. back : 38
10. more : 38
11. ever : 37
12. too : 32
13. again : 32
14. only : 31
15. always : 29
16. just : 27
17. there : 27
18. where : 26
19. down : 26
20. once : 25
21. still : 24
22. also : 24
23. away : 24
24. later : 23
25. almost : 23
26. well : 22
27. as : 20
28. When : 19
29. already : 18
30. soon : 18
31. together : 18
32. Then : 18
33. how : 17
34. suddenly : 17
35. up : 16
36. enough : 16
37. Even : 16
38. yet : 14
39. immediately : 13
40. far : 12
41. forward : 12
42. however : 11
43. much : 11
44. most : 10
45. quite : 9
46. nearly : 9
47. Only : 9
48. before : 9
49. sometimes : 9
50. finally : 8


In [7]:
analyze.top_k_words(50, pos_tags=['ADJ']) # top 50 aggettivi più frequenti

1. other : 60
2. own : 40
3. human : 39
4. old : 38
5. more : 31
6. good : 29
7. long : 28
8. little : 28
9. few : 28
10. great : 27
11. such : 26
12. last : 25
13. first : 24
14. same : 21
15. better : 19
16. whole : 18
17. hard : 18
18. many : 17
19. young : 16
20. certain : 16
21. white : 15
22. able : 15
23. terrible : 15
24. big : 14
25. much : 14
26. small : 13
27. tremendous : 13
28. usual : 13
29. animal : 13
30. full : 13
31. necessary : 13
32. bad : 13
33. large : 12
34. next : 11
35. black : 11
36. late : 11
37. green : 11
38. dead : 9
39. past : 9
40. right : 9
41. single : 8
42. early : 8
43. secret : 8
44. new : 8
45. several : 8
46. strange : 7
47. different : 7
48. short : 7
49. free : 7
50. happy : 7


In [8]:
analyze.top_k_ngrams(20, 1) # top 20 unigrammi di parole più frequenti

1. ('the',) : 2020
2. (',',) : 1938
3. ('.',) : 1461
4. ('and',) : 903
5. ('of',) : 898
6. ('to',) : 803
7. ('was',) : 633
8. ('a',) : 584
9. ('had',) : 528
10. ('in',) : 499
11. ('that',) : 437
12. ('it',) : 308
13. ('were',) : 290
14. ('they',) : 277
15. ('his',) : 260
16. ('animals',) : 243
17. ('he',) : 242
18. ('for',) : 241
19. ('on',) : 233
20. ('with',) : 220


In [9]:
analyze.top_k_ngrams(20, 2) # top 20 bigrammi di parole più frequenti

1. (',', 'and') : 316
2. ('of', 'the') : 257
3. ('in', 'the') : 179
4. ('.', 'The') : 177
5. ('the', 'animals') : 135
6. (',', 'the') : 134
7. ('had', 'been') : 109
8. ('to', 'the') : 107
9. ('it', 'was') : 100
10. ('and', 'the') : 89
11. ('the', 'farm') : 84
12. (',', 'but') : 83
13. ('on', 'the') : 83
14. ('.', 'It') : 79
15. ('.', '``') : 76
16. (',', 'he') : 71
17. ('.', 'He') : 71
18. ('to', 'be') : 67
19. ('that', 'the') : 65
20. ('.', 'They') : 65


In [10]:
analyze.top_k_ngrams(20, 3) # top 20 trigrammi di parole più frequenti

1. (',', 'and', 'the') : 51
2. ('.', 'It', 'was') : 49
3. ('.', 'The', 'animals') : 39
4. (',', 'it', 'was') : 31
5. ('he', 'said', ',') : 30
6. (',', 'he', 'said') : 26
7. ('on', 'the', 'farm') : 24
8. ('of', 'the', 'farm') : 24
9. (',', 'comrades', ',') : 21
10. ('.', 'They', 'were') : 19
11. ('the', 'animals', 'were') : 18
12. ("'Beasts", 'of', 'England') : 18
13. ('of', 'England', "'") : 18
14. ('of', 'the', 'animals') : 18
15. ('there', 'was', 'a') : 17
16. ('the', 'other', 'animals') : 17
17. ('the', 'farm', ',') : 17
18. ('that', 'they', 'had') : 16
19. ('all', 'the', 'animals') : 16
20. ('.', 'In', 'the') : 16


In [11]:
analyze.top_k_ngrams(20, 4) # top 20 quadrigrammi di parole più frequenti

1. (',', 'he', 'said', ',') : 23
2. ("'Beasts", 'of', 'England', "'") : 18
3. ('the', 'Battle', 'of', 'the') : 13
4. ('Four', 'legs', 'good', ',') : 12
5. ('legs', 'good', ',', 'two') : 12
6. ('good', ',', 'two', 'legs') : 12
7. ('Battle', 'of', 'the', 'Cowshed') : 12
8. (',', "''", 'he', 'said') : 10
9. ('.', 'The', 'animals', 'were') : 10
10. ('from', 'side', 'to', 'side') : 9
11. (',', 'and', 'it', 'was') : 9
12. (',', 'two', 'legs', 'bad') : 9
13. ('on', 'the', 'farm', ',') : 8
14. ('Jones', 'and', 'his', 'men') : 7
15. ('.', 'It', 'was', 'a') : 7
16. ('of', "'Beasts", 'of', 'England') : 7
17. ('the', 'farm', 'buildings', ',') : 7
18. ('the', 'big', 'barn', ',') : 6
19. ('the', 'farm', ',', 'and') : 6
20. ('the', 'animals', 'on', 'the') : 6


In [12]:
analyze.top_k_ngrams(20, 5) # top 20 quinquigrammi di parole più frequenti

1. ('Four', 'legs', 'good', ',', 'two') : 12
2. ('legs', 'good', ',', 'two', 'legs') : 12
3. ('the', 'Battle', 'of', 'the', 'Cowshed') : 12
4. ('good', ',', 'two', 'legs', 'bad') : 9
5. ('of', "'Beasts", 'of', 'England', "'") : 7
6. (',', "''", 'he', 'said', ',') : 6
7. ("''", 'he', 'said', ',', '``') : 6
8. ('``', 'I', 'will', 'work', 'harder') : 6
9. ('``', 'Four', 'legs', 'good', ',') : 6
10. ('back', 'to', 'the', 'farm', 'buildings') : 5
11. (',', 'two', 'legs', 'bad', '!') : 5
12. ('Battle', 'of', 'the', 'Cowshed', ',') : 5
13. ('at', 'the', 'Battle', 'of', 'the') : 5
14. ('from', 'side', 'to', 'side', ',') : 4
15. ('of', 'the', 'big', 'barn', ',') : 4
16. ('.', '``', 'Comrades', ',', "''") : 4
17. ("'Beasts", 'of', 'England', "'", '.') : 4
18. ('at', 'the', 'foot', 'of', 'the') : 4
19. ('singing', 'of', "'Beasts", 'of', 'England') : 4
20. ("'Beasts", 'of', 'England', "'", ',') : 4


In [13]:
analyze.top_k_ngrams(20, 1, pos=True) # top 20 unigrammi di pos più frequenti

1. ('NOUN',) : 7179
2. ('VERB',) : 6241
3. ('.',) : 4184
4. ('ADP',) : 3936
5. ('DET',) : 3932
6. ('PRON',) : 2474
7. ('ADV',) : 1994
8. ('ADJ',) : 1856
9. ('CONJ',) : 1223
10. ('PRT',) : 1183
11. ('NUM',) : 239
12. ('X',) : 12


In [14]:
analyze.top_k_ngrams(20, 2, pos=True) # top 20 bigrammi di pos più frequenti

1. ('DET', 'NOUN') : 2498
2. ('NOUN', '.') : 2312
3. ('ADP', 'DET') : 1619
4. ('NOUN', 'ADP') : 1525
5. ('NOUN', 'VERB') : 1346
6. ('VERB', 'VERB') : 1325
7. ('PRON', 'VERB') : 1293
8. ('ADJ', 'NOUN') : 1117
9. ('VERB', 'ADP') : 966
10. ('ADP', 'NOUN') : 943
11. ('VERB', 'ADV') : 828
12. ('DET', 'ADJ') : 718
13. ('VERB', 'DET') : 709
14. ('.', 'PRON') : 673
15. ('ADP', 'PRON') : 668
16. ('.', 'DET') : 647
17. ('.', 'CONJ') : 608
18. ('.', 'NOUN') : 558
19. ('VERB', 'PRON') : 552
20. ('VERB', '.') : 543


In [15]:
analyze.top_k_ngrams(20, 3, pos=True) # top 20 trigrammi di pos più frequenti

1. ('ADP', 'DET', 'NOUN') : 1153
2. ('DET', 'NOUN', 'ADP') : 676
3. ('DET', 'NOUN', '.') : 648
4. ('NOUN', 'ADP', 'DET') : 608
5. ('DET', 'ADJ', 'NOUN') : 599
6. ('.', 'PRON', 'VERB') : 580
7. ('DET', 'NOUN', 'VERB') : 543
8. ('NOUN', 'ADP', 'NOUN') : 487
9. ('NOUN', 'VERB', 'VERB') : 439
10. ('VERB', 'DET', 'NOUN') : 424
11. ('ADJ', 'NOUN', '.') : 419
12. ('PRON', 'VERB', 'VERB') : 410
13. ('NOUN', '.', 'DET') : 386
14. ('VERB', 'ADP', 'DET') : 384
15. ('NOUN', '.', 'CONJ') : 365
16. ('.', 'DET', 'NOUN') : 353
17. ('NOUN', '.', 'PRON') : 344
18. ('VERB', 'VERB', 'ADP') : 336
19. ('NOUN', '.', 'NOUN') : 309
20. ('ADP', 'DET', 'ADJ') : 307


In [16]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN')) # top 10 bigrammi (composti da aggettivo e sostantivo) per frequenza

1. ('human', 'beings') : 23
2. ('other', 'animals') : 20
3. ('big', 'barn') : 10
4. ('first', 'time') : 8
5. ('other', 'animal') : 7
6. ('same', 'time') : 6
7. ('old', 'Major') : 5
8. ('few', 'minutes') : 5
9. ('whole', 'farm') : 5
10. ('next', 'moment') : 5


In [17]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN'), order='cond_prob') # top 10 bigrammi (composti da aggettivo e sostantivo) per probabiità condizionata

1. ('handsome', 'money') : 1.0
2. ('reasonable', 'price') : 1.0
3. ('sordid', 'labour') : 1.0
4. ('majestic-looking', 'pig') : 1.0
5. ('admirable', 'care') : 1.0
6. ('sick', 'comrade') : 1.0
7. ('sidelong', 'look') : 1.0
8. ('false', 'step') : 1.0
9. ('turnip', 'field') : 1.0
10. ('individual', 'letters') : 1.0


In [18]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN'), order='joint_prob') # top 10 bigrammi (composti da aggettivo e sostantivo) per probabiità congiunta

1. ('human', 'beings') : 0.000667576118189998
2. ('other', 'animals') : 0.0005805009723391287
3. ('big', 'barn') : 0.00029025048616956433
4. ('first', 'time') : 0.00023220038893565146
5. ('other', 'animal') : 0.00020317534031869504
6. ('same', 'time') : 0.0001741502917017386
7. ('young', 'pigs') : 0.00014512524308478217
8. ('next', 'moment') : 0.00014512524308478217
9. ('outside', 'world') : 0.00014512524308478217
10. ('five-barred', 'gate') : 0.00014512524308478217


In [19]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN'), order='mi') # top 10 bigrammi (composti da aggettivo e sostantivo) per mutua informazione

1. ('false', 'step') : 20.01927051117619
2. ('nesting', 'boxes') : 20.01927051117619
3. ('swift', 'dash') : 20.01927051117619
4. ('unforeseen', 'shortages') : 20.01927051117619
5. ('momentary', 'awkwardness') : 20.01927051117619
6. ('fourth', 'foal') : 20.01927051117619
7. ('rash', 'actions') : 20.01927051117619
8. ('Own', 'Bricklayer') : 20.01927051117619
9. ('deepest', 'distress') : 20.01927051117619
10. ('basic', 'slag') : 20.01927051117619


In [20]:
analyze.top_k_bigrams(10, pos_tags=('ADJ', 'NOUN'), order='lmi') # top 10 bigrammi (composti da aggettivo e sostantivo) per mutua informazione locale

1. ('human', 'beings') : 337.2195256508365
2. ('other', 'animals') : 210.18991013698508
3. ('big', 'barn') : 148.89987494231224
4. ('first', 'time') : 94.17074398186084
5. ('five-barred', 'gate') : 86.05957794559293
6. ('outside', 'world') : 79.16701982932429
7. ('other', 'animal') : 77.60791376422374
8. ('same', 'time') : 71.78392845405
9. ('few', 'minutes') : 70.37206032684325
10. ('next', 'moment') : 67.44724782323749


In [21]:
analyze.common_mi_lmi(10, ('ADJ', 'NOUN'))

Non ci sono bigrammi in comune nei primi 10 bigrammi con pos ('ADJ', 'NOUN') ordinati per mutual information e local mutual information.


In [22]:
analyze.analyze_sentences()

Frase con la media della distribuzione di frequenza dei token più alta (899.25): "After the horses came Muriel,
the white goat, and Benjamin, the donkey.".
Frase con la media della distribuzione di frequenza dei token più bassa (53.11): "Others asked such questions as "Why should we care what
happens after we are dead?"".
Frase con la probabilità più alta secondo un modello di Markov di ordine 2 (6.472902553197826e-30): "It was a pig walking on his hind legs.".


In [23]:
analyze.ner(15) # top 15 NE più frequenti per ciascuna classe di NE con relativa frequenza

PERSON
1. Napoleon : 112
2. Boxer : 73
3. Squealer : 65
4. Snowball : 63
5. Jones : 46
6. Animal Farm : 37
7. Benjamin : 31
8. Clover : 26
9. Frederick : 24
10. Mr. Jones : 22
11. Mollie : 20
12. Muriel : 14
13. Mr. Pilkington : 14
14. Major : 10
15. Foxwood : 9

GPE
1. England : 28
2. Napoleon : 22
3. Snowball : 18
4. Boxer : 13
5. Jones : 12
6. Clover : 9
7. Animalism : 8
8. Pilkington : 7
9. Willingdon : 5
10. Long : 5
11. Whymper : 5
12. Frederick : 4
13. Foxwood : 3
14. Jessie : 2
15. Ireland : 2

ORGANIZATION
1. Snowball : 27
2. Comrade Napoleon : 24
3. Rebellion : 16
4. Battle : 13
5. Seven Commandments : 10
6. Cowshed : 8
7. Bluebell : 3
8. Republic : 3
9. Willingdon : 3
10. Sundays : 3
11. Commandment : 3
12. Berkshire : 2
13. Red Lion : 2
14. Animals : 2
15. Second Class : 2

LOCATION
1. Clean Tails League : 1

FACILITY
1. Crown Derby : 2
2. Fifth Commandment : 1

